<a href="https://colab.research.google.com/github/cu7th0n/ssq/blob/master/ssq_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import xlwt
import time

#获取第一页的内容
def get_one_page(url):
    headers = {
        'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36'
    }
    response = requests.get(url,headers=headers)
    if response.status_code == 200:
        return response.text
    return 

#解析第一页内容，数据结构化
def parse_one_page(html):

    soup = BeautifulSoup(html,'lxml')
    i = 0
    for item in soup.select('tr')[2:-1]:

        yield{
            'time':item.select('td')[i].text,
            'digit_1':item.select('td em')[0].text,
            'digit_2':item.select('td em')[1].text,
            'digit_3':item.select('td em')[2].text,
            'digit_4':item.select('td em')[3].text,
            'digit_5':item.select('td em')[4].text,
            'digit_6':item.select('td em')[5].text,
            'digit_7':item.select('td em')[6].text,
        }

#将数据写入Excel表格中
def write_to_excel():
    f = xlwt.Workbook()                             
    sheet1 = f.add_sheet('ssq',cell_overwrite_ok=True)
    row0 = ["date","digit_1","digit_2","digit_3","digit_4","digit_5","digit_6","digit_7"]
    #写入第一行
    for j in range(0,len(row0)):
        sheet1.write(0,j,row0[j])

    #依次爬取每一页内容的每一期信息，并将其依次写入Excel
    i=0
    for k in range(1,120):
        url = 'http://kaijiang.zhcw.com/zhcw/html/ssq/list_%s.html' %(str(k))
        html = get_one_page(url)
        
        #写入每一期的信息
        for item in parse_one_page(html):
            sheet1.write(i+1,0,item['time'])
            sheet1.write(i+1,1,item['digit_1'])
            sheet1.write(i+1,2,item['digit_2'])
            sheet1.write(i+1,3,item['digit_3'])
            sheet1.write(i+1,4,item['digit_4'])
            sheet1.write(i+1,5,item['digit_5'])
            sheet1.write(i+1,6,item['digit_6'])
            sheet1.write(i+1,7,item['digit_7'])
            i+=1
    
    f.save('ssq.xls')
    print('%d页已保存。'%k)
    
def main():
    write_to_excel()

if __name__ == '__main__':
    main()

119页已保存。


In [0]:
import pandas as pd
data = pd.read_excel('ssq.xls')

data['date'] = pd.to_datetime(data['date'])
data = data.sort_values(by = 'date')
data.reset_index(inplace=True)
del data['index']
del data['date']

In [0]:
D_1 = data['digit_1']
D_2 = data['digit_2']
D_3 = data['digit_3']
D_4 = data['digit_4']
D_5 = data['digit_5']
D_6 = data['digit_6']
D_7 = data['digit_7']

In [7]:
from keras.models import load_model
from keras.utils import to_categorical
import numpy as np
models = ['M1_model.h5', 'M2_model.h5', 'M3_model.h5', 'M4_model.h5', 'M5_model.h5','M6_model.h5','M7_model.h5']
tdatas = [D_1,D_2,D_3,D_4,D_5,D_6,D_7]

for (model,tdata) in zip(models,tdatas):
    
    print(model)
    M_ssq = load_model(model)

    test_X = to_categorical(np.asarray([tdata[-200:]]),num_classes=34)

    pred = M_ssq.predict(test_X)
    ranks = np.argsort(pred[0])
    
    for i in range(9):
        print(str(ranks[-i-1]) + ' : %.2f%%' %(pred[0][ranks[-i-1]]*100))
    print('\n')

M1_model.h5
1 : 18.89%
2 : 16.17%
5 : 11.96%
4 : 9.57%
6 : 8.21%
3 : 7.99%
7 : 6.56%
9 : 6.30%
8 : 5.33%


M2_model.h5
8 : 12.67%
6 : 9.34%
7 : 8.26%
13 : 7.04%
4 : 6.70%
11 : 6.64%
10 : 6.35%
5 : 6.20%
9 : 5.78%


M3_model.h5
12 : 8.14%
14 : 8.01%
15 : 7.76%
8 : 7.51%
16 : 7.16%
10 : 6.24%
11 : 5.25%
19 : 5.24%
9 : 5.10%


M4_model.h5
20 : 8.24%
24 : 7.41%
22 : 7.13%
17 : 6.88%
23 : 6.31%
14 : 5.80%
19 : 5.70%
26 : 5.68%
15 : 5.12%


M5_model.h5
27 : 11.57%
25 : 9.52%
23 : 7.50%
26 : 6.82%
28 : 6.77%
22 : 6.75%
32 : 5.63%
29 : 5.25%
24 : 4.90%


M6_model.h5
33 : 17.35%
32 : 16.41%
29 : 12.64%
30 : 12.45%
31 : 9.98%
27 : 7.17%
28 : 5.47%
26 : 4.97%
25 : 4.35%


M7_model.h5
7 : 12.84%
15 : 9.56%
5 : 9.46%
10 : 9.27%
13 : 7.83%
12 : 6.76%
9 : 5.68%
8 : 5.48%
14 : 5.44%


